In [2]:
import requests
import json
import csv
import pandas as pd
import matplotlib as plt
import numpy as np
from config import api_key

ImportError: cannot import name 'api_key' from 'config' (/Users/Tmunson50/opt/anaconda3/lib/python3.8/site-packages/config/__init__.py)

In [3]:
csvfile = "./Resources/2019.csv"
csv_df = pd.read_csv(csvfile)
csv_df["Year"] = "2019"
len(csv_df)

620

In [4]:
csvfile18 = "./Resources/2018.csv"
csv_df18 = pd.read_csv(csvfile18)
csv_df18["Year"] = "2018"
len(csv_df18)

622

In [5]:
csvfile17 = "./Resources/2017.csv"
csv_df17 = pd.read_csv(csvfile17)
csv_df17["Year"] = "2017"
len(csv_df17)

571

In [6]:
csvfile16 = "./Resources/2016.csv"
csv_df16 = pd.read_csv(csvfile16)
csv_df16["Year"] = "2016"
len(csv_df16)

584

In [7]:
csvfile15 = "./Resources/2015.csv"
csv_df15 = pd.read_csv(csvfile15)
csv_df15["Year"] = "2015"
len(csv_df15)

595

In [8]:
combined_df = csv_df.append(csv_df18)
combined_df = combined_df.append(csv_df17)
combined_df = combined_df.append(csv_df16)
combined_df = combined_df.append(csv_df15)
combined_df.drop("Unnamed: 0", axis=1, inplace=True)

In [32]:
playercsv = "./Resources/player_data.csv"
player_df = pd.read_csv(playercsv)
merged_df = pd.merge(combined_df, player_df, left_on = "Player", right_on = "Name", how="left")
merged_df.loc[merged_df["Name"]=="Dak Prescott"]

,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,Int,...,PlayerID,Name,Height,HeightFeet,HeightInches,Weight,Experience,Position,ADP,Salary
13,Dak Prescott,DAL,QB,26.0,16.0,16.0,388.0,596.0,4902.0,11.0,...,18055.0,Dak Prescott,"6'2""",6.0,2.0,238.0,7.0,QB,58.4,NaN
697,Dak Prescott,DAL,QB,25.0,16.0,16.0,356.0,526.0,3885.0,8.0,...,18055.0,Dak Prescott,"6'2""",6.0,2.0,238.0,7.0,QB,58.4,NaN
1326,Dak Prescott,DAL,QB,24.0,16.0,16.0,308.0,490.0,3324.0,13.0,...,18055.0,Dak Prescott,"6'2""",6.0,2.0,238.0,7.0,QB,58.4,NaN
1878,Dak Prescott,DAL,QB,23.0,16.0,16.0,311.0,459.0,3667.0,4.0,...,18055.0,Dak Prescott,"6'2""",6.0,2.0,238.0,7.0,QB,58.4,NaN


In [30]:
merged_df.columns

Index(['Player', 'Tm', 'Pos', 'Age', 'G', 'GS', 'Cmp', 'Att', 'Yds', 'Int',
       'Att.1', 'Yds.1', 'Tgt', 'Rec', 'Yds.2', 'Y/R', 'Fumbles',
       'FumblesLost', 'PassingYds', 'PassingTD', 'PassingAtt', 'RushingYds',
       'RushingTD', 'RushingAtt', 'ReceivingYds', 'ReceivingTD',
       'FantasyPoints', 'Year', 'Unnamed: 0', 'PlayerID', 'Name', 'Height',
       'HeightFeet', 'HeightInches', 'Weight', 'Experience', 'Position', 'ADP',
       'Salary'],
      dtype='object')

In [9]:
combined_df.to_csv("Combined_DF_Data")